In [1]:
from skilift import GTFS, get_stop_node, read_osm, get_elevations_for_nodes, get_graph_nodes, ElevationAwareStreetDataset

In [2]:
import pandas as pd
from zipfile import ZipFile
from collections import defaultdict
from typing import Dict, Set, Tuple, Iterable, Iterator, Any
import numpy as np
from shapely.geometry import Point, LineString, MultiLineString
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
fn = "/home/bmander/skilift_data/transit/redding.zip"

feed = GTFS(fn)

In [4]:
# get epoch time on Aptil 19, 2023 at 1:00pm pacific time
t0 = pd.Timestamp("2023-04-19 13:00:00", tz="America/Los_Angeles")
t0

Timestamp('2023-04-19 13:00:00-0700', tz='America/Los_Angeles')

In [5]:
node0 = get_stop_node(feed, "Downtown Transit Center", t0)
node0

AtStopNode(stop_id:2000, datetime:2023-04-19 13:00:00-07:00)

In [6]:
node0.outgoing[0].node

DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:0, datetime:2023-04-19 14:25:00-07:00)

In [7]:
node0.outgoing[0].node.outgoing[0].node

ArrivalNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00)

In [8]:
node0.outgoing[0].node.outgoing[0].node.outgoing

[Edge(node=DepartureNode(pattern_id:5, service_id:c_1658_b_18260_d_31, row:1, col:1, datetime:2023-04-19 14:35:00-07:00), weight=0.0),
 Edge(node=AtStopNode(stop_id:8003, datetime:2023-04-19 14:35:00-07:00), weight=60.0)]

In [9]:
osm_filename = "/home/bmander/skilift_data/street/redding.pbf"


In [10]:
elev_geotiff_filename = "/home/bmander/skilift_data/elevation/USGS_13_n41w123_20210624.tif"

In [11]:
elev_dataset = ElevationAwareStreetDataset(osm_filename, elev_geotiff_filename)

Reading OSM file...done
Indexing ways...done
Getting node elevations...done
Creating spatial index...done


In [24]:
def geodesic_distance_meters(geo_pt: Point, geo_pt2: Point) -> float:
    """Compute the geodesic distance in meters between two points on the earth's surface.
    """

    # TODO: this can be vectorized if it ever becomes a bottleneck

    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [geo_pt.x, geo_pt.y, geo_pt2.x, geo_pt2.y])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    earth_radius_km = 6371
    km = earth_radius_km * c
    return km * 1000

In [25]:
nd_points = elev_dataset.nodes.values()
lon = sum([x[0] for x in nd_points])/len(nd_points)
lat = sum([x[1] for x in nd_points])/len(nd_points)
lon, lat

(40.58422925237579, -122.35866061888524)

In [26]:
way_id, seg_ix, linear_ref = elev_dataset.get_nearest_segment(lat, lon)
way_id, seg_ix, linear_ref

(10557590, 13, 0.18715448062975482)